In [ ]:
using Pkg
pkg"activate ."

In [ ]:
# Run this cell in order to download all the package dependencies with the exact versions used in the book
# This is necessary if (some of) the packages have been updated and have introduced breaking changes
pkg"instantiate"

In [ ]:
using CSV, DataFrames

In [ ]:
df = CSV.read("data/une_rt_m_1.tsv", header = true, delim = '\t')

In [ ]:
values = convert(Array, df[1, 2:end])

In [ ]:
values = map(x -> parse(Int, replace(x, " "=>"")), values)[:]

In [ ]:
dates = names(df)[2:end]

In [ ]:
dates = map(x -> replace(string(x), "M"=>"-"), dates)

In [ ]:
using Dates
dateformat = DateFormat("y-m")

In [ ]:
dates = map(x -> Date(x, dateformat), dates)

In [ ]:
df2 = DataFrame(Dates = dates, Values = values)

In [ ]:
CSV.write("UE-unemployment.tsv", df2)

In [ ]:
using TimeSeries
unemployment_data = readtimearray("UE-unemployment.tsv")

In [ ]:
TimeSeries.head(unemployment_data, 10)

In [ ]:
using Pkg
pkg"add PyCall LaTeXStrings"

In [ ]:
using Plots
pyplot()

In [ ]:
plot(unemployment_data)

In [ ]:
plot()
for y in 2005:2017
	TimeSeries.values(when(unemployment_data, year, y))[:] |> plot!
    gui()
end

In [ ]:
using Statistics
moving_avg = moving(mean, unemployment_data, 12)

In [ ]:
moving(mean, unemployment_data, 12, padding = true)

In [ ]:
plot(unemployment_data)
plot!(moving_avg)

In [ ]:
sn = unemployment_data ./ moving_avg

In [ ]:
plot(sn)

In [ ]:
month_avg = Float64[]

In [ ]:
for m in 1:12
	md = when(sn, month, m)
	push!(month_avg, mean(TimeSeries.values(md)[:]))
end

In [ ]:
month_avg

In [ ]:
s = sum(month_avg)

In [ ]:
norm_month_avg = map(m -> 12m/s, month_avg)

In [ ]:
sum(norm_month_avg)

In [ ]:
adj_unemployment_data = deepcopy(unemployment_data)

In [ ]:
map(adj_unemployment_data) do d,v
	v[1] /= norm_month_avg[month(d)]
	d,v
end

In [ ]:
using Dates
ts1 = TimeArray(Date(today()) - Week(1):Day(1):Date(today()) |> collect, rand(8))

In [ ]:
ts2 = TimeArray(Date(today()) - Week(1):Day(1):Date(today()) |> collect, rand(8))

In [ ]:
tsc = ts1 .> ts2

In [ ]:
tsc .== false

In [ ]:
diffts = diff(unemployment_data)

In [ ]:
bar(diffts)

In [ ]:
ts1

In [ ]:
lag(ts1)

In [ ]:
lagged = lag(unemployment_data, 12)

In [ ]:
common = merge(unemployment_data, lagged)

In [ ]:
plot(unemployment_data)
plot!(lagged)

In [ ]:
update(unemployment_data, Date(2018, 1, 1), TimeSeries.values(adj_unemployment_data[end])[:][end] * norm_month_avg[1] |> round)

In [ ]:
mean(TimeSeries.values(adj_unemployment_data)[:])

In [ ]:
findall(adj_unemployment_data[:Values] .== maximum(TimeSeries.values(adj_unemployment_data)[:]))

In [ ]:
adj_unemployment_data[98]

In [ ]:
last_trend = from(adj_unemployment_data, Date(2013, 2, 1))

In [ ]:
x = 1:length(last_trend)

In [ ]:
y = TimeSeries.values(last_trend)[:]

In [ ]:
linreg(x, y) = reverse([x ones(length(x))]\y)
a, b = linreg(x, y)

In [ ]:
y = a+b*60

In [ ]:
y = y * norm_month_avg[1] |> round

In [ ]:
update(unemployment_data, Date(2018, 1, 1), y) |> plot
plot!(unemployment_data)